In [1]:
import os
os.chdir(r"C:\Users\Laura\Desktop\saeb_microdados_dashboard")

In [ ]:

import pandas as pd
from ingestion.read_alunos95 import read_alunos
from dictionaries.alunos_maps import (
    map_categorias_basicas_alunos,
    map_geografia_alunos,
    map_questionario_socioeconomico,
    map_opiniao_matematica,
    map_data_nascimento_visual
    
)
from cleaning.alunos_cleaning95 import (
    clean_alunos_data,
    drop_missing_essential,
    clean_zero_to_nan,
)

In [3]:
df_alunos = read_alunos(1995)
pd.set_option("display.max_rows", None)

df_alunos = map_categorias_basicas_alunos(df_alunos)
df_alunos = map_geografia_alunos(df_alunos)
df_alunos = map_questionario_socioeconomico(df_alunos)
df_alunos = map_opiniao_matematica(df_alunos)
df_alunos = map_data_nascimento_visual(df_alunos)
df_alunos = clean_alunos_data(df_alunos)
df_alunos = drop_missing_essential(df_alunos)
df_alunos = clean_zero_to_nan(df_alunos)


In [5]:
df_alunos.head()

,ano,serie,id_aluno,idade,estrato,upa,dep_adm,local,rede,uf,regiao,peso_ac,proficiencia,data_nascimento,sexo,raca,instrucao_pai,instrucao_mae,mora_com,trabalha,compra_jornal,horas_tv,conversa_escola,estudo_interrompido,faltas,anos_1grau,anos_2grau,supletivo,opiniao_teste,tempo_licao,gosta_mat,entende_professor,genero_mat,utilidade_mat,esforco_professor
0,1995,3º Ano,15,20.0,110,1,Municipal,Urbana,Pública,Rondônia,Norte,26.0,204.7948,09/04/1975,Feminino,Pardo(a),Nunca frequentou,Nunca frequentou,1.0,Sim (8h/dia),1.0,2.0,2.0,3.0,5 dias,1.0,2.0,2.0,Muito difícil,1.0,Concordo em parte,Concordo em parte,Concordo totalmente,Concordo totalmente,Não sei
1,1995,3º Ano,10,24.0,110,2,Municipal,Urbana,Pública,Rondônia,Norte,37.0,218.7780,18/02/1971,Masculino,Preto(a),Primário (1ª-4ª),Primário (1ª-4ª),2.0,Sim (8h/dia),5.0,3.0,2.0,5.0,> 30 dias,2.0,1.0,2.0,Difícil,3.0,Concordo em parte,Discordo em parte,Concordo em parte,Concordo totalmente,Concordo em parte
2,1995,3º Ano,27,18.0,110,6,Federal,Urbana,Particular,Rondônia,Norte,14.0,329.1833,28/08/1976,Feminino,Preto(a),Superior,Ginásio (5ª-8ª),1.0,Não,4.0,2.0,2.0,1.0,15 dias,1.0,3.0,2.0,Mais ou menos,5.0,Concordo totalmente,Concordo em parte,NaN,Concordo em parte,Concordo em parte
3,1995,3º Ano,9,NaN,120,3,Municipal,Urbana,Pública,Acre,Norte,10.0,210.8740,Não Informado,Feminino,Pardo(a),Primário (1ª-4ª),Primário (1ª-4ª),5.0,Sim (4-6h/dia),5.0,4.0,2.0,1.0,5 dias,1.0,3.0,2.0,Muito difícil,1.0,Concordo totalmente,Não sei,Concordo totalmente,Concordo totalmente,NaN
4,1995,3º Ano,31,17.0,120,4,Federal,Urbana,Particular,Acre,Norte,9.0,309.3175,03/12/1977,Feminino,Pardo(a),Primário (1ª-4ª),Primário (1ª-4ª),2.0,Não,5.0,3.0,2.0,1.0,5 dias,1.0,3.0,2.0,Mais ou menos,4.0,Concordo em parte,Discordo em parte,Não sei,Discordo em parte,Não sei


In [ ]:
df_alunos[df_alunos['uf']=="Acre"]['dep_adm'].value_counts()

In [ ]:
df_alunos.info()

In [ ]:
import plotly.express as px
import streamlit as st

# 1. Agrupar os dados pela média de proficiência
df_comparativo = df_alunos.groupby("dep_adm")["proficiencia"].mean().reset_index()

# 2. Ordenar para o gráfico ficar mais intuitivo (opcional)
df_comparativo = df_comparativo.sort_values("proficiencia", ascending=False)

In [ ]:
# Criando a figura
fig = px.bar(
    df_comparativo, 
    x="dep_adm", 
    y="proficiencia",
    title="Média de Proficiência por Dependência Administrativa",
    labels={"dep_adm": "Dependência Adm.", "proficiencia": "Proficiência Média"},
    color="proficiencia", # Muda a cor baseado no valor
    color_continuous_scale="Viridis",
    text_auto='.1f' # Exibe o valor em cima da barra com 1 casa decimal
)

# Ajustando o layout
fig.update_layout(xaxis_title="Rede de Ensino", yaxis_title="Proficiência (Média)")

# Exibindo no Streamlit
st.plotly_chart(fig, use_container_width=True)

In [ ]:
ufs = st.multiselect("Selecione os Estados", options=df_alunos['uf'].unique())
if ufs:
    df_filtrado = df_alunos[df_alunos['uf'].isin(ufs)]
else:
    df_filtrado = df_alunos